TODO:

1. 檢查是不是能透過kernelSHAP得到vi

In [1]:
import shap
import os
import time
import pandas as pd
import numpy as np
import json
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from data_process import read_selected_data, get_y,  split_data, compute_class_weights
from dataset import BertDataset
from model import BertClassifier
from training import train_model
from utils import draw_pics, initial_record
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import matplotlib.pyplot as plt
total_params = 14
csv_file_path = '/hcds_vol/private/luffy/GANGAN-master/data/processed_data/v014_stage_1.csv'
json_file_path = '/hcds_vol/private/luffy/GANGAN-master/data/controllable_para_v014_14.json'
tool_name = 'ASCVD'
epochs = 50000
lr = 1e-5
batch_size = 1024
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
save_folder_name = 'stage-1-param_'+str(total_params)+'-batch_'+str(batch_size)+'-lr_'+str(lr)
with open(json_file_path, 'r') as f:
    params = json.load(f)
    f.close()
s1_df = pd.read_csv(csv_file_path)
#s1_df.shape
all_key = list(params[tool_name]) 
params_list = []                #取得json檔內的特徵
for key in all_key:
    all_param = params[tool_name][key]
    if(type(all_param) == list):
        for param in all_param:
             params_list.append(param)
    else:
        params_list.append(all_param)
    
# 取得Json檔內包含的特徵
s1_df = s1_df[params_list] 
#print(params)
s1_df.head(10)
feature_df = s1_df.drop(['DFT_CNT'], axis=1)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
scaler = StandardScaler()
X_standardized = scaler.fit_transform(feature_df)
nf_df = pd.DataFrame(X_standardized)
nf_df
param_group = [] # [2,2,4,2]
all_key = list(params[tool_name]) # ['EQ', 'PUMP', 'CH', 'VENT', 'y']
all_key.remove('y')

for key in all_key:
    all_value = params[tool_name][key]
    param_group.append(len(all_value))
param_group


[2, 2, 8, 2]

In [3]:
def padding_zero(df, tool_name, total_params, flag, params=params): 
    # 將一維參數matrix擴展為4維
    data_arr = df.to_numpy()
    result = []
    for i in range(len(data_arr)):
        arr_index = 0
        empty_arr = np.zeros((4,total_params)) # chamber數 * 總參數數量
        param_group_cp = param_group.copy()
        for j in range(len(empty_arr)):
            while(param_group_cp[j] > 0):
                empty_arr[j][arr_index] = data_arr[i][arr_index]
                param_group_cp[j] -= 1
                arr_index += 1
        
        if(flag == 1): # bert.py使用
            result.append(empty_arr)
        if(flag == 2): # bert_du.py使用
            result.append(empty_arr.tolist())
    
    if(flag == 1): # bert.py使用
        result = pd.DataFrame({'X': [result[i] for i in range(len(result))]})
    return result

In [4]:
nf_df_4d = padding_zero(nf_df,tool_name,total_params,flag=1)
nf_df_4d_object = nf_df_4d.to_numpy()
nf_df_4d_list = []
for i in range(len(nf_df_4d_object)):
    nf_df_4d_list.append(nf_df_4d_object[i][0])
nf_df_4d_arr = np.array(nf_df_4d_list)

In [5]:
from torch.utils.data import DataLoader, TensorDataset
s1_model_path = '/hcds_vol/private/luffy/GANGAN-master/model/predictor/stage_1_checkpoint.pth'
s1_model =  torch.load(s1_model_path).to(device)
s1_model.eval()
nf_df_4d_tensor = torch.tensor(nf_df_4d_arr,dtype=torch.float)
dataset = TensorDataset(nf_df_4d_tensor)
batch_size = 256
loader = DataLoader(dataset, batch_size=batch_size)
outputs = []
with torch.no_grad():
    for batch_data in loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
        batch_data = batch_data[0].to(device)
        
        # 将数据传递给模型进行推理
        batch_output = s1_model(batch_data)
        probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
        outputs += probs

In [6]:

output_arr = np.array([output.cpu().numpy()[0] for output in outputs])
output_df = pd.DataFrame({'Output': output_arr})
new_df = pd.concat([feature_df,output_df],axis=1)

In [7]:
def send_to_model(data):
    #if 'Output' in data.columns:
         #data = data.drop(columns=['Output'])
    data_standardized = scaler.transform(data)
    data_standardized_df = pd.DataFrame(data_standardized)
    data_4d = padding_zero(data_standardized_df,tool_name,total_params,flag=1)
    data_4d_object =  data_4d.to_numpy()
    data_4d_list = []
    for i in range(len(data_4d_object)):
        data_4d_list.append(data_4d_object[i][0])
    data_4d_arr = np.array(data_4d_list)
    data_4d_tensor = torch.tensor(data_4d_arr,dtype=torch.float)
    my_dataset = TensorDataset(data_4d_tensor)
    batch_size = 256
    my_loader = DataLoader(my_dataset, batch_size=batch_size,num_workers=4)
    data_output = []
    with torch.no_grad():
        for batch_data in my_loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
            batch_data = batch_data[0].to(device)
        
        # 将数据传递给模型进行推理
            batch_output = s1_model(batch_data)
            probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
            data_output += probs
    data_output_arr = np.array([output.cpu().numpy()[0] for output in data_output])
    #data_expectation_out = data_output_arr.mean()
    return data_output_arr

In [8]:
import numpy as np 
import warnings
import itertools
# 過濾掉FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)
def all_subsets(n_elts):
    '''
        returns a list of 2^{n_elts} lists
        each a different subset of {1, 2,...,n_elts}
    '''
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]
para_num = 14
AUO_coalitions = [np.array([])] + all_subsets(para_num) 
coalition_estimated_values = {}
instance = new_df.iloc[2]
#mean_exp = new_df['Output'].mean()
flag=0
count_n = 1
selected_data = new_df.copy()
selected_data = selected_data[(selected_data['PUMP_low']<20000) & 
                              (selected_data['PUMP_high']>20000) & 
                              (selected_data['VENT_low']<10000) & 
                              (selected_data['VENT_high']>10000) &
                              (selected_data['NH3_TREAT_-RF_FREQ-max']>13800)&
                              (selected_data['NH3_TREAT_-RF_FREQ-mean']<13800)
                                                            ]
sample_df = selected_data.sample(n=1000,random_state=42)
print(type(sample_df))
mean_exp = sample_df['Output'].mean()
sample_df = sample_df.drop(columns=['Output'])
instance = instance.drop(labels=['Output'])
explainer = shap.KernelExplainer(send_to_model, sample_df)
shap_values = explainer.shap_values(instance, nsamples=1000)


<class 'pandas.core.frame.DataFrame'>


X does not have valid feature names, but StandardScaler was fitted with feature names


Using 1000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
X does not have valid feature names, but StandardScaler was fitted with feature names


做堆疊
1000
[[1.50000000e+02 3.15000000e+03 2.73519833e+04 ... 1.00000000e+00
  1.68432882e+04 7.87221111e+03]
 [1.60000000e+02 2.65000000e+03 5.40493333e+04 ... 1.00000000e+00
  1.95210769e+04 8.40490000e+03]
 [1.50000000e+02 3.15000000e+03 4.13653600e+04 ... 1.00000000e+00
  1.49724937e+04 5.65047857e+03]
 ...
 [1.40000000e+02 3.15000000e+03 3.68338400e+04 ... 1.00000000e+00
  1.59592867e+04 7.13825000e+03]
 [1.60000000e+02 2.65000000e+03 4.58784200e+04 ... 1.00000000e+00
  1.91068385e+04 7.90180000e+03]
 [1.45000000e+02 3.15000000e+03 5.07670000e+04 ... 1.00000000e+00
  1.63849875e+04 7.31258182e+03]]
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
here7
h

X does not have valid feature names, but StandardScaler was fitted with feature names


(1000, 13)
(1000,)


In [9]:
shap.plots.waterfall(shap_values[0])

TypeError: The waterfall plot requires an `Explanation` object as the `shap_values` argument.

In [ ]:
print("shap_values =", shap_values)

shap_values = [ 0.09619054 -0.02663675 -0.01717055 -0.02686515 -0.02065319 -0.06259205
 -0.00205657  0.0096463  -0.01979932 -0.02788637 -0.00862422 -0.00307333
 -0.00796304  0.01788066]


In [ ]:
print(params_list)

['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean', 'PUMP_high', 'PUMP_low', 'CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', 'NH3_TREAT_-RF_FREQ-mean', 'NP_3_-MFC_VOL_SIH4-range', 'VENT_high', 'VENT_low', 'DFT_CNT']
